In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras import datasets
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.math import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
import os

for dir in os.listdir("/kaggle/input/plant-village/PlantVillage/"):
    print(dir)

In [4]:
import random
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
%matplotlib inline

plt.figure(figsize=(50,30))
test_folder=r'/kaggle/input/plant-village/PlantVillage/Tomato_healthy'
for i in range(5):
    file = random.choice(os.listdir(test_folder))
    image_path= os.path.join(test_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

In [6]:
tomato=[
"Tomato_healthy",
"Tomato_Bacterial_spot",
"Tomato__Target_Spot",]

In [7]:
import PIL
from PIL import Image

In [8]:
def create_dataset_PIL(img_folder):

    img_data_array=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        if(dir1 in tomato):
            try:
                for file in os.listdir(os.path.join(img_folder, dir1)):

                    image_path= os.path.join(img_folder, dir1, file)
                    image= np.array(Image.open(image_path))
                    img_data_array.append(image)
                    class_name.append(dir1)
            except:
                pass
    return img_data_array , class_name



In [9]:
PIL_img_data, class_name=create_dataset_PIL("/kaggle/input/plant-village/PlantVillage/")

In [10]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_dict

In [11]:
class_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(PIL_img_data, class_val,test_size = 0.2,random_state = 41)

In [13]:
x_train=np.array(x_train)
x_train.shape

In [14]:
x_test=np.array(x_test)
x_test.shape

In [15]:
y_train=np.array(y_train)
y_train.shape

In [16]:
y_test=np.array(y_test)
y_test.shape

In [17]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [18]:
y_train.shape

In [19]:
y_test.shape

In [20]:
x_train = x_train/255

x_test = x_test/255

In [21]:
x_train=x_train.reshape((x_train.shape[0],-1))
print("Train set shape after reshaping {}".format(x_train.shape))

In [22]:
x_test = x_test.reshape((x_test.shape[0], -1))
print("Test set shape after reshaping {}".format(x_test.shape))

In [24]:
m1=Sequential()

In [25]:
m1.add(Dense(10, input_shape = (196608,), activation = 'sigmoid',kernel_initializer = initializers.HeUniform))
m1.add(Dense(20,activation='relu',kernel_initializer = initializers.HeUniform))
m1.add(Dense(20,activation='relu',kernel_initializer = initializers.HeUniform))
m1.add(Dense(30,activation='relu',kernel_initializer = initializers.HeUniform))
m1.add(Dense(20,activation='relu',kernel_initializer = initializers.HeUniform))
m1.add(Dense(3,activation='softmax',kernel_initializer = initializers.HeUniform))
m1.summary()

In [26]:
sgd = optimizers.SGD(learning_rate = 0.01)    # stochastic gradient descent optimizer
m1.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [27]:
history=m1.fit(x_train, y_train, batch_size = 50, epochs = 5, verbose = 1,validation_split=0.3)

In [28]:
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Train_accuracy','val_accuracy'],loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train_loss','val_loss'],loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [31]:
from tensorflow.keras.layers import BatchNormalization

m2=Sequential()

m2.add(Dense(10, input_shape = (196608,), activation = 'sigmoid'))

m2.add(BatchNormalization(momentum=0.9,beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones'))

m2.add(Dense(20,activation='relu'))

m2.add(BatchNormalization(momentum=0.9,beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones'))

m2.add(Dense(20,activation='relu'))

m2.add(BatchNormalization(momentum=0.7,beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones'))

m2.add(Dense(30,activation='relu'))

m2.add(BatchNormalization(momentum=0.7,beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones'))

m2.add(Dense(3,activation='softmax'))
m2.summary()

In [32]:
m2.compile(optimizer ='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [33]:
history2=m2.fit(x_train, y_train, batch_size = 50, epochs = 5, verbose = 1,validation_split=0.3)

In [34]:
from keras.layers import Dropout

In [35]:
m3=Sequential()

m3.add(Dense(10, input_shape = (196608,), activation = 'sigmoid'))
m3.add(Dense(20,activation='relu'))

m3.add(Dropout(0.2))

m3.add(Dense(20,activation='relu'))

m3.add(Dropout(0.2))

m3.add(Dense(30,activation='relu'))

m3.add(Dropout(0.3))

m3.add(Dense(3,activation='softmax'))
m3.summary()


In [36]:
m3.compile(optimizer ='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history3=m3.fit(x_train, y_train, batch_size = 50, epochs = 5, verbose = 1,validation_split=0.3)

In [ ]:
m4=Sequential()

m4.add(Dense(10, input_shape = (196608,), activation = 'sigmoid'))
m4.add(Dense(20,activation='relu'))
m4.add(Dense(20,activation='relu'))
m4.add(Dense(30,activation='relu'))
m4.add(Dense(3,activation='softmax'))
m4.summary()

In [ ]:
m4.compile(optimizer ='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
import tensorflow as tf


callback = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)


history4=m4.fit(x_train, y_train, batch_size = 50, epochs = 5, verbose = 1,validation_split=0.3,callbacks=[callback])